In [2]:
%pip install pandas
%pip install numpy


  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 42.9 MB/s eta 0:00:00 0:00:01
Using cached tzdata-2025.1-py2.py3-none-any.whl (346 kB)

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [42]:
import pandas as pd
import numpy as np
from datetime import datetime

In [4]:
subset= pd.read_csv('second_model_subset.csv')
features = pd.read_csv('second_models_features.csv')

/var/folders/kb/s8dqbsbd69z1lpcq7967w6g40000gn/T/ipykernel_80149/2332485499.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  subset= pd.read_csv('second_model_subset.csv')


In [34]:
path = 'chest-xrays/'
segmented = pd.read_csv(path+"CXLSeg-segmented.csv")
xray = pd.read_csv(path+'CXLSeg-metadata.csv')



In [40]:
def date_format(date):
    date = str(date)
    formatted_date = f"{date[:4]}-{date[4:6]}-{date[6:]}"
    return formatted_date

def time_format(time):
    time = str(time)
    time = time.split(".")[0]
    while len(time) != 6:
        time = "0" + time
    formatted_time = f"{time[:2]}:{time[2:4]}:{time[4:6]}"
    return formatted_time

def convert_datetime(input_date):
    return datetime.fromisoformat(input_date)





In [43]:
subset["admittime"] = subset["admittime"].apply(convert_datetime)
subset["dischtime"] = subset["dischtime"].apply(convert_datetime)

xray = xray.assign(formatted_date = xray["StudyDate"].apply(date_format))
xray = xray.assign(formatted_time = xray["StudyTime"].apply(time_format))
xray = xray.assign(studytime = (xray["formatted_date"] + " " + xray["formatted_time"]).apply(convert_datetime))

In [44]:
# Subsetting xray dataset to make merge more efficient
xray_merge = xray[["subject_id", "study_id", "ViewPosition", "studytime"]]
# First merge
merging = subset.merge(xray_merge, left_on = "subject_id", right_on = "subject_id")
# Matching each xray to hospital admission
matched_dates = merging[(merging["studytime"] >= merging["admittime"]) & (merging["studytime"] <= merging["dischtime"])].reset_index(drop = True)
# Preprocessing segmented for merging
segmented_merged = segmented[["subject_id", "study_id", "dicom_id", "DicomPath", "No Finding"]]
segmented_merged["No Finding"] = segmented_merged["No Finding"].fillna(-1)
segmented_merged["Abnormal"] = (segmented_merged["No Finding"] * -1)
segmented_merged = segmented_merged.drop(columns = ["No Finding"])
# Final merge
complete_merged = matched_dates.merge(segmented_merged, on = ["subject_id", "study_id"])[["subject_id", "hadm_id", "stay_id", "study_id", 
                                                                       "admittime", "dischtime", "studytime", "ViewPosition",
                                                                       "dicom_id", "DicomPath", "Abnormal", "los", 
                                                                       "chronic_pulmonary_disease", "sepsis3"]]

complete_merged

/var/folders/kb/s8dqbsbd69z1lpcq7967w6g40000gn/T/ipykernel_80149/604059279.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  segmented_merged["No Finding"] = segmented_merged["No Finding"].fillna(-1)
/var/folders/kb/s8dqbsbd69z1lpcq7967w6g40000gn/T/ipykernel_80149/604059279.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  segmented_merged["Abnormal"] = (segmented_merged["No Finding"] * -1)


,subject_id,hadm_id,stay_id,study_id,admittime,dischtime,studytime,ViewPosition,dicom_id,DicomPath,Abnormal,los,chronic_pulmonary_disease,sepsis3
0,12207593,22795209,30000646,55490538,2194-04-27 18:43:00,2194-05-06 02:29:00,2194-05-04 08:28:30,AP,e076666b-e2e941fe-7206948e-96d77b1f-790ebb2c,files/p12/p12207593/s55490538/e076666b-e2e941f...,1.0,8 days 07:46:00,0,True
1,12207593,22795209,30000646,56369155,2194-04-27 18:43:00,2194-05-06 02:29:00,2194-04-30 08:33:53,AP,7f124ef7-a1714162-b7e9da18-5862d297-3fd8f3a9,files/p12/p12207593/s56369155/7f124ef7-a171416...,1.0,8 days 07:46:00,0,True
2,12207593,22795209,30000646,57440284,2194-04-27 18:43:00,2194-05-06 02:29:00,2194-04-29 06:25:10,AP,49d77ea6-5f9a013f-d0e6c33e-bb3aae40-88d57b43,files/p12/p12207593/s57440284/49d77ea6-5f9a013...,1.0,8 days 07:46:00,0,True
3,12207593,22795209,30000646,57520242,2194-04-27 18:43:00,2194-05-06 02:29:00,2194-05-02 10:45:39,AP,40f24d85-94293a52-b7e4961d-d2ad9561-864efe77,files/p12/p12207593/s57520242/40f24d85-94293a5...,-1.0,8 days 07:46:00,0,True
4,12207593,22795209,30000646,57688955,2194-04-27 18:43:00,2194-05-06 02:29:00,2194-05-01 08:06:11,AP,f725e388-3b557432-84d5882b-4c406d73-68132627,files/p12/p12207593/s57688955/f725e388-3b55743...,-1.0,8 days 07:46:00,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135876,13096583,20322847,35254911,57452898,2167-07-30 18:01:00,2167-08-10 12:30:00,2167-08-05 10:21:56,AP,291d4cd9-78036eac-9f682833-a7cf1fcb-b05106fc,files/p13/p13096583/s57452898/291d4cd9-78036ea...,1.0,10 days 18:29:00,0,False
135877,15754398,24924327,30970584,52232141,2150-09-03 06:30:00,2150-09-11 14:09:00,2150-09-03 08:25:34,AP,49851e5c-9eaa60d4-0bb0bc52-d3b30ef0-6bded685,files/p15/p15754398/s52232141/49851e5c-9eaa60d...,-1.0,8 days 07:39:00,0,False
135878,15754398,24924327,30970584,52871799,2150-09-03 06:30:00,2150-09-11 14:09:00,2150-09-03 20:07:27,AP,19a40edf-c3e21c44-93e035a6-caf1d6dc-f22a8105,files/p15/p15754398/s52871799/19a40edf-c3e21c4...,1.0,8 days 07:39:00,0,False
135879,15754398,24924327,30970584,58185683,2150-09-03 06:30:00,2150-09-11 14:09:00,2150-09-04 04:55:36,AP,71dc37d3-41be0418-b5590a13-ad7d107e-425ab985,files/p15/p15754398/s58185683/71dc37d3-41be041...,1.0,8 days 07:39:00,0,False


In [138]:
features = pd.read_csv('second_models_features.csv')
features = features[features['subject_id'].notna()]

In [179]:
# sub = features[features['subject_id']==16192578.0]
# sub.sort_values(by = 'charttime', ascending = False)

# sub.merge(sub.groupby(['subject_id', 'hadm_id', 'stay_id'])[['charttime']].max().reset_index(), on = 'charttime').columns
len(features.columns)

19

In [174]:
# recents = features.groupby(['subject_id', 'hadm_id', 'stay_id'])[['charttime']].max().reset_index()
# features.merge(recents, on = ['subject_id', 'hadm_id', 'stay_id', 'charttime'])
recents = features.sort_values(['subject_id', 'hadm_id', 'stay_id', 'charttime']).groupby(['subject_id', 'hadm_id', 'stay_id']).tail(1)
# recents
recents = recents.reset_index().drop(columns = 'index')
recents.iloc[0]

subject_id              10001217.0
hadm_id                   24597018
stay_id                 37067082.0
charttime      2157-11-21 03:16:00
heart_rate                     NaN
sbp                            NaN
sbp_ni                         NaN
mbp                            NaN
mbp_ni                         NaN
resp_rate                      NaN
temperature                    NaN
platelet                     285.0
wbc                           19.0
bands                          NaN
lactate                        NaN
inr                            1.2
ptt                           33.1
creatinine                     0.4
bilirubin                      NaN
Name: 0, dtype: object

In [176]:
means = features.groupby(['subject_id', 'hadm_id', 'stay_id'])[['heart_rate', 'sbp',
       'sbp_ni', 'mbp', 'mbp_ni', 'resp_rate', 'temperature', 'platelet',
       'wbc', 'bands', 'lactate', 'inr', 'ptt', 'creatinine', 'bilirubin']].mean().reset_index()
means.iloc[0]

subject_id     10001217.0
hadm_id        24597018.0
stay_id        37067082.0
heart_rate            NaN
sbp                   NaN
sbp_ni                NaN
mbp                   NaN
mbp_ni                NaN
resp_rate             NaN
temperature           NaN
platelet            285.0
wbc                  19.0
bands                 NaN
lactate               NaN
inr                   1.2
ptt                  33.1
creatinine            0.4
bilirubin             NaN
Name: 0, dtype: float64

In [191]:
feat_squeeze = recents.combine_first(means)
feat_squeeze.notna().sum()/(feat_squeeze.isna().sum()+feat_squeeze.notna().sum())




bands          0.121041
bilirubin      0.479033
charttime      1.000000
creatinine     0.998375
hadm_id        1.000000
heart_rate     0.275407
inr            0.840090
lactate        0.034608
mbp            0.227159
mbp_ni         0.125331
platelet       0.998290
ptt            0.832763
resp_rate      0.273639
sbp            0.226403
sbp_ni         0.124947
stay_id        1.000000
subject_id     1.000000
temperature    0.110322
wbc            0.998546
dtype: float64

In [183]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 355200 entries, 0 to 1955734
Data columns (total 19 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   subject_id   355200 non-null  float64
 1   hadm_id      355200 non-null  int64  
 2   stay_id      355200 non-null  float64
 3   charttime    355200 non-null  object 
 4   heart_rate   30890 non-null   float64
 5   sbp          24777 non-null   float64
 6   sbp_ni       10962 non-null   float64
 7   mbp          24984 non-null   float64
 8   mbp_ni       11013 non-null   float64
 9   resp_rate    30620 non-null   float64
 10  temperature  10485 non-null   float64
 11  platelet     342561 non-null  float64
 12  wbc          341343 non-null  float64
 13  bands        18253 non-null   float64
 14  lactate      2686 non-null    float64
 15  inr          224663 non-null  float64
 16  ptt          221344 non-null  float64
 17  creatinine   341758 non-null  float64
 18  bilirubin    117628 non-null

In [48]:
full_data = complete_merged.merge(features, how = 'left', on = ['subject_id', 'hadm_id'])
full_data.head()

,subject_id,hadm_id,stay_id_x,study_id,admittime,dischtime,studytime,ViewPosition,dicom_id,DicomPath,...,resp_rate,temperature,platelet,wbc,bands,lactate,inr,ptt,creatinine,bilirubin
0,12207593,22795209,30000646,55490538,2194-04-27 18:43:00,2194-05-06 02:29:00,2194-05-04 08:28:30,AP,e076666b-e2e941fe-7206948e-96d77b1f-790ebb2c,files/p12/p12207593/s55490538/e076666b-e2e941f...,...,NaN,NaN,171.0,9.1,NaN,NaN,NaN,NaN,0.5,NaN
1,12207593,22795209,30000646,55490538,2194-04-27 18:43:00,2194-05-06 02:29:00,2194-05-04 08:28:30,AP,e076666b-e2e941fe-7206948e-96d77b1f-790ebb2c,files/p12/p12207593/s55490538/e076666b-e2e941f...,...,NaN,NaN,230.0,7.2,NaN,NaN,1.6,32.9,NaN,NaN
2,12207593,22795209,30000646,55490538,2194-04-27 18:43:00,2194-05-06 02:29:00,2194-05-04 08:28:30,AP,e076666b-e2e941fe-7206948e-96d77b1f-790ebb2c,files/p12/p12207593/s55490538/e076666b-e2e941f...,...,30.0,NaN,189.0,8.8,NaN,NaN,1.4,37.2,0.6,NaN
3,12207593,22795209,30000646,55490538,2194-04-27 18:43:00,2194-05-06 02:29:00,2194-05-04 08:28:30,AP,e076666b-e2e941fe-7206948e-96d77b1f-790ebb2c,files/p12/p12207593/s55490538/e076666b-e2e941f...,...,NaN,NaN,266.0,7.9,NaN,NaN,NaN,NaN,0.6,NaN
4,12207593,22795209,30000646,55490538,2194-04-27 18:43:00,2194-05-06 02:29:00,2194-05-04 08:28:30,AP,e076666b-e2e941fe-7206948e-96d77b1f-790ebb2c,files/p12/p12207593/s55490538/e076666b-e2e941f...,...,NaN,NaN,337.0,10.6,NaN,NaN,1.3,35.3,1.0,0.7


In [49]:
full_data.columns

Index(['subject_id', 'hadm_id', 'stay_id_x', 'study_id', 'admittime',
       'dischtime', 'studytime', 'ViewPosition', 'dicom_id', 'DicomPath',
       'Abnormal', 'los', 'chronic_pulmonary_disease', 'sepsis3', 'stay_id_y',
       'charttime', 'heart_rate', 'sbp', 'sbp_ni', 'mbp', 'mbp_ni',
       'resp_rate', 'temperature', 'platelet', 'wbc', 'bands', 'lactate',
       'inr', 'ptt', 'creatinine', 'bilirubin'],
      dtype='object')

In [54]:
full_data = full_data[(full_data['stay_id_y'].notna())]
full_data

,subject_id,hadm_id,stay_id_x,study_id,admittime,dischtime,studytime,ViewPosition,dicom_id,DicomPath,...,resp_rate,temperature,platelet,wbc,bands,lactate,inr,ptt,creatinine,bilirubin
0,12207593,22795209,30000646,55490538,2194-04-27 18:43:00,2194-05-06 02:29:00,2194-05-04 08:28:30,AP,e076666b-e2e941fe-7206948e-96d77b1f-790ebb2c,files/p12/p12207593/s55490538/e076666b-e2e941f...,...,NaN,NaN,171.0,9.1,NaN,NaN,NaN,NaN,0.5,NaN
1,12207593,22795209,30000646,55490538,2194-04-27 18:43:00,2194-05-06 02:29:00,2194-05-04 08:28:30,AP,e076666b-e2e941fe-7206948e-96d77b1f-790ebb2c,files/p12/p12207593/s55490538/e076666b-e2e941f...,...,NaN,NaN,230.0,7.2,NaN,NaN,1.6,32.9,NaN,NaN
2,12207593,22795209,30000646,55490538,2194-04-27 18:43:00,2194-05-06 02:29:00,2194-05-04 08:28:30,AP,e076666b-e2e941fe-7206948e-96d77b1f-790ebb2c,files/p12/p12207593/s55490538/e076666b-e2e941f...,...,30.0,NaN,189.0,8.8,NaN,NaN,1.4,37.2,0.6,NaN
3,12207593,22795209,30000646,55490538,2194-04-27 18:43:00,2194-05-06 02:29:00,2194-05-04 08:28:30,AP,e076666b-e2e941fe-7206948e-96d77b1f-790ebb2c,files/p12/p12207593/s55490538/e076666b-e2e941f...,...,NaN,NaN,266.0,7.9,NaN,NaN,NaN,NaN,0.6,NaN
4,12207593,22795209,30000646,55490538,2194-04-27 18:43:00,2194-05-06 02:29:00,2194-05-04 08:28:30,AP,e076666b-e2e941fe-7206948e-96d77b1f-790ebb2c,files/p12/p12207593/s55490538/e076666b-e2e941f...,...,NaN,NaN,337.0,10.6,NaN,NaN,1.3,35.3,1.0,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2639634,15754398,24924327,30970584,58185683,2150-09-03 06:30:00,2150-09-11 14:09:00,2150-09-04 04:55:36,AP,71dc37d3-41be0418-b5590a13-ad7d107e-425ab985,files/p15/p15754398/s58185683/71dc37d3-41be041...,...,NaN,NaN,113.0,11.3,NaN,NaN,1.2,27.5,1.2,NaN
2639635,15754398,24924327,30970584,58210659,2150-09-03 06:30:00,2150-09-11 14:09:00,2150-09-04 11:47:53,AP,54a0e380-53080f0b-f48fee21-e82fa8aa-db7354ac,files/p15/p15754398/s58210659/54a0e380-53080f0...,...,NaN,NaN,92.0,8.5,NaN,NaN,NaN,NaN,0.5,NaN
2639636,15754398,24924327,30970584,58210659,2150-09-03 06:30:00,2150-09-11 14:09:00,2150-09-04 11:47:53,AP,54a0e380-53080f0b-f48fee21-e82fa8aa-db7354ac,files/p15/p15754398/s58210659/54a0e380-53080f0...,...,NaN,NaN,94.0,7.1,NaN,NaN,NaN,NaN,0.7,NaN
2639637,15754398,24924327,30970584,58210659,2150-09-03 06:30:00,2150-09-11 14:09:00,2150-09-04 11:47:53,AP,54a0e380-53080f0b-f48fee21-e82fa8aa-db7354ac,files/p15/p15754398/s58210659/54a0e380-53080f0...,...,NaN,NaN,106.0,8.8,NaN,NaN,NaN,NaN,0.8,NaN


In [57]:
full_data.columns
full_data[['subject_id', 'hadm_id', 'stay_id_x', 'study_id', 'admittime',
       'dischtime', 'studytime', 'ViewPosition', 'dicom_id', 'charttime']]

,subject_id,hadm_id,stay_id_x,study_id,admittime,dischtime,studytime,ViewPosition,dicom_id,charttime
0,12207593,22795209,30000646,55490538,2194-04-27 18:43:00,2194-05-06 02:29:00,2194-05-04 08:28:30,AP,e076666b-e2e941fe-7206948e-96d77b1f-790ebb2c,2194-05-03 03:59:00
1,12207593,22795209,30000646,55490538,2194-04-27 18:43:00,2194-05-06 02:29:00,2194-05-04 08:28:30,AP,e076666b-e2e941fe-7206948e-96d77b1f-790ebb2c,2194-04-30 05:30:00
2,12207593,22795209,30000646,55490538,2194-04-27 18:43:00,2194-05-06 02:29:00,2194-05-04 08:28:30,AP,e076666b-e2e941fe-7206948e-96d77b1f-790ebb2c,2194-05-01 09:00:00
3,12207593,22795209,30000646,55490538,2194-04-27 18:43:00,2194-05-06 02:29:00,2194-05-04 08:28:30,AP,e076666b-e2e941fe-7206948e-96d77b1f-790ebb2c,2194-04-29 19:33:00
4,12207593,22795209,30000646,55490538,2194-04-27 18:43:00,2194-05-06 02:29:00,2194-05-04 08:28:30,AP,e076666b-e2e941fe-7206948e-96d77b1f-790ebb2c,2194-04-29 06:09:00
...,...,...,...,...,...,...,...,...,...,...
2639634,15754398,24924327,30970584,58185683,2150-09-03 06:30:00,2150-09-11 14:09:00,2150-09-04 04:55:36,AP,71dc37d3-41be0418-b5590a13-ad7d107e-425ab985,2150-09-03 07:43:00
2639635,15754398,24924327,30970584,58210659,2150-09-03 06:30:00,2150-09-11 14:09:00,2150-09-04 11:47:53,AP,54a0e380-53080f0b-f48fee21-e82fa8aa-db7354ac,2150-09-05 02:50:00
2639636,15754398,24924327,30970584,58210659,2150-09-03 06:30:00,2150-09-11 14:09:00,2150-09-04 11:47:53,AP,54a0e380-53080f0b-f48fee21-e82fa8aa-db7354ac,2150-09-03 16:26:00
2639637,15754398,24924327,30970584,58210659,2150-09-03 06:30:00,2150-09-11 14:09:00,2150-09-04 11:47:53,AP,54a0e380-53080f0b-f48fee21-e82fa8aa-db7354ac,2150-09-04 01:55:00


In [97]:
sample = full_data[full_data['subject_id']==12207593]
sample['stay_id_x'].value_counts()
sample = sample[['subject_id', 'hadm_id', 'stay_id_x', 'study_id', 'admittime',
       'dischtime', 'studytime', 'ViewPosition', 'dicom_id', 'charttime']]
sample['charttime'] = sample['charttime'].apply(convert_datetime)

In [98]:
sample[sample['stay_id_x']==30000646]

sample[sample['stay_id_x']==30000646]['study_id'].value_counts()

(sample[sample['study_id']==55490538]['charttime']-sample[sample['study_id']==55490538]['studytime']).sort_values(ascending=False)

0         -2 days +19:30:30
1942889   -2 days +19:30:30
5         -3 days +21:11:30
1942894   -3 days +21:11:30
2         -3 days +00:31:30
1942891   -3 days +00:31:30
1         -5 days +21:01:30
1942890   -5 days +21:01:30
3         -5 days +11:04:30
1942892   -5 days +11:04:30
4         -6 days +21:40:30
1942893   -6 days +21:40:30
6         -6 days +18:04:30
1942895   -6 days +18:04:30
dtype: timedelta64[ns]

In [99]:
sample['days'] = abs(sample['studytime']-sample['charttime'])

sample

,subject_id,hadm_id,stay_id_x,study_id,admittime,dischtime,studytime,ViewPosition,dicom_id,charttime,days
0,12207593,22795209,30000646,55490538,2194-04-27 18:43:00,2194-05-06 02:29:00,2194-05-04 08:28:30,AP,e076666b-e2e941fe-7206948e-96d77b1f-790ebb2c,2194-05-03 03:59:00,1 days 04:29:30
1,12207593,22795209,30000646,55490538,2194-04-27 18:43:00,2194-05-06 02:29:00,2194-05-04 08:28:30,AP,e076666b-e2e941fe-7206948e-96d77b1f-790ebb2c,2194-04-30 05:30:00,4 days 02:58:30
2,12207593,22795209,30000646,55490538,2194-04-27 18:43:00,2194-05-06 02:29:00,2194-05-04 08:28:30,AP,e076666b-e2e941fe-7206948e-96d77b1f-790ebb2c,2194-05-01 09:00:00,2 days 23:28:30
3,12207593,22795209,30000646,55490538,2194-04-27 18:43:00,2194-05-06 02:29:00,2194-05-04 08:28:30,AP,e076666b-e2e941fe-7206948e-96d77b1f-790ebb2c,2194-04-29 19:33:00,4 days 12:55:30
4,12207593,22795209,30000646,55490538,2194-04-27 18:43:00,2194-05-06 02:29:00,2194-05-04 08:28:30,AP,e076666b-e2e941fe-7206948e-96d77b1f-790ebb2c,2194-04-29 06:09:00,5 days 02:19:30
...,...,...,...,...,...,...,...,...,...,...,...
1942919,12207593,22795209,39582132,57688955,2194-04-27 18:43:00,2194-05-06 02:29:00,2194-05-01 08:06:11,AP,f725e388-3b557432-84d5882b-4c406d73-68132627,2194-05-01 09:00:00,0 days 00:53:49
1942920,12207593,22795209,39582132,57688955,2194-04-27 18:43:00,2194-05-06 02:29:00,2194-05-01 08:06:11,AP,f725e388-3b557432-84d5882b-4c406d73-68132627,2194-04-29 19:33:00,1 days 12:33:11
1942921,12207593,22795209,39582132,57688955,2194-04-27 18:43:00,2194-05-06 02:29:00,2194-05-01 08:06:11,AP,f725e388-3b557432-84d5882b-4c406d73-68132627,2194-04-29 06:09:00,2 days 01:57:11
1942922,12207593,22795209,39582132,57688955,2194-04-27 18:43:00,2194-05-06 02:29:00,2194-05-01 08:06:11,AP,f725e388-3b557432-84d5882b-4c406d73-68132627,2194-05-02 05:40:00,0 days 21:33:49


In [108]:
# for category, group in sample.groupby(['hadm_id', 'study_id']):
#     print(f"Category: {category}")
#     print(group, "\n")

sample.merge(sample.groupby(['hadm_id', 'study_id'])[['days']].reset_index(), on = 'days')

AttributeError: 'DataFrameGroupBy' object has no attribute 'reset_index'

In [114]:
sample.groupby(['hadm_id', 'study_id'])[['days']].min().reset_index()

,hadm_id,study_id,days
0,22795209,55490538,1 days 04:29:30
1,22795209,56369155,0 days 03:03:53
2,22795209,57440284,0 days 00:16:10
3,22795209,57520242,0 days 05:05:39
4,22795209,57688955,0 days 00:53:49
